In [5]:
import pandas as pd
import wmfdata as wmf
from wmfdata.charting import set_mpl_style
from wmfdata.utils import pd_display_all

In [6]:
set_mpl_style()

# Extracting KaiOS data from webrequest

In [ ]:
wmf.hive.run("""
CREATE EXTERNAL TABLE `neilpquinn.kaios_wp_webrequest`(
  `hostname` string COMMENT 'Source node hostname', 
  `sequence` bigint COMMENT 'Per host sequence number', 
  `dt` string COMMENT 'Timestame at cache in ISO 8601', 
  `time_firstbyte` double COMMENT 'Time to first byte', 
  `ip` string COMMENT 'IP of packet at cache', 
  `cache_status` string COMMENT 'Cache status', 
  `http_status` string COMMENT 'HTTP status of response', 
  `response_size` bigint COMMENT 'Response size', 
  `http_method` string COMMENT 'HTTP method of request', 
  `uri_host` string COMMENT 'Host of request', 
  `uri_path` string COMMENT 'Path of request', 
  `uri_query` string COMMENT 'Query of request', 
  `content_type` string COMMENT 'Content-Type header of response', 
  `referer` string COMMENT 'Referer header of request', 
  `x_forwarded_for` string COMMENT 'X-Forwarded-For header of request', 
  `user_agent` string COMMENT 'User-Agent header of request', 
  `accept_language` string COMMENT 'Accept-Language header of request', 
  `x_analytics` string COMMENT 'X-Analytics header of response', 
  `range` string COMMENT 'Range header of response', 
  `is_pageview` boolean COMMENT 'Indicates if this record was marked as a pageview during refinement', 
  `record_version` string COMMENT 'Keeps track of changes in the table content definition - https://wikitech.wikimedia.org/wiki/Analytics/Data/Webrequest', 
  `client_ip` string COMMENT 'Client IP computed during refinement using ip and x_forwarded_for', 
  `geocoded_data` map<string,string> COMMENT 'Geocoded map with continent, country_code, country, city, subdivision, postal_code, latitude, longitude, timezone keys and associated values', 
  `x_cache` string COMMENT 'X-Cache header of response', 
  `user_agent_map` map<string,string> COMMENT 'User-agent map with browser_family, browser_major, device_family, os_family, os_major, os_minor and wmf_app_version keys and associated values', 
  `x_analytics_map` map<string,string> COMMENT 'X_analytics map view of the x_analytics field', 
  `ts` timestamp COMMENT 'Unix timestamp in milliseconds extracted from dt', 
  `access_method` string COMMENT 'Method used to access the site (mobile app|mobile web|desktop)', 
  `agent_type` string COMMENT 'Categorise the agent making the webrequest as either user or spider (automatas to be added).', 
  `is_zero` boolean COMMENT 'Indicates if the webrequest is accessed through a zero provider', 
  `referer_class` string COMMENT 'Indicates if a referer is internal, external or unknown.', 
  `normalized_host` struct<project_class:string,project:string,qualifiers:array<string>,tld:string,project_family:string> COMMENT 'struct containing project_family (such as wikipedia or wikidata for instance), project (such as en or commons), qualifiers (a list of in-between values, such as m and/or zero) and tld (org most often)', 
  `pageview_info` map<string,string> COMMENT 'map containing project, language_variant and page_title values only when is_pageview = TRUE.', 
  `page_id` int COMMENT 'MediaWiki page_id for this page title. For redirects this could be the page_id of the redirect or the page_id of the target. This may not always be set, even if the page is actually a pageview.', 
  `namespace_id` int COMMENT 'MediaWiki namespace_id for this page title. This may not always be set, even if the page is actually a pageview.', 
  `tags` array<string> COMMENT 'List containing tags qualifying the request, ex: [portal, wikidata]. Will be used to split webrequest into smaller subsets.', 
  `isp_data` map<string,string> COMMENT 'Internet Service Provider data in a map with keys isp, organization, autonomous_system_organization and autonomous_system_number', 
  `accept` string COMMENT 'Accept header of request', 
  `tls` string COMMENT 'TLS information of request', 
  `tls_map` map<string,string> COMMENT 'Map view of TLS information (keys are vers, keyx, auth and ciph)',
  `webrequest_source` string COMMENT 'Source cluster' 
)
PARTITIONED BY ( 
  `year` int COMMENT 'Unpadded year of request', 
  `month` int COMMENT 'Unpadded month of request', 
  `day` int COMMENT 'Unpadded day of request' 
)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
""")

In [22]:
# Only days 1 to 30, for a 30-day month
for day in range(1, 31):
  wmf.hive.run(
    """
    INSERT INTO TABLE neilpquinn.kaios_wp_webrequest
    PARTITION (year = 2020, month = 7, day = {day})
    SELECT
      hostname,
      sequence,
      dt,
      time_firstbyte,
      ip,
      cache_status,
      http_status,
      response_size,
      http_method,
      uri_host,
      uri_path,
      uri_query,
      content_type,
      referer,
      x_forwarded_for,
      user_agent,
      accept_language,
      x_analytics,
      range,
      is_pageview,
      record_version,
      client_ip,
      geocoded_data,
      x_cache,
      user_agent_map,
      x_analytics_map,
      ts,
      access_method,
      agent_type,
      is_zero,
      referer_class,
      normalized_host,
      pageview_info,
      page_id,
      namespace_id,
      tags,
      isp_data,
      accept,
      tls,
      tls_map,
      webrequest_source
    FROM wmf.webrequest
    WHERE
      LOWER(user_agent) LIKE "%kaios%" AND
      normalized_host.project_family = "wikipedia" AND
      year = 2020 AND month = 10 AND day = {day}
    """.format(day=day)
  )

In [27]:
wmf.hive.run("""
  SELECT
    year,
    month,
    day,
    COUNT(*) as records
  FROM neilpquinn.kaios_wp_webrequest
  WHERE
    year > 0
  GROUP BY
    year,
    month,
    day
  ORDER BY
    year,
    month,
    day
  LIMIT 1000000
""")

,year,month,day,records
0,2020,7,1,2973568
1,2020,7,2,3023231
2,2020,7,3,2999027
3,2020,7,4,3138759
4,2020,7,5,3251344
5,2020,7,6,3125497
6,2020,7,7,3187281
7,2020,7,8,3128749
8,2020,7,9,3245367
9,2020,7,10,3055885


# Geographic location

In [34]:
pageviews_by_country = hive.run("""
SELECT
  geocoded_data["country"] AS country,
  COUNT(1) as pageviews
FROM neilpquinn.kaios_wp_webrequest
WHERE
  agent_type = 'user' AND
  is_pageview AND
  webrequest_source = 'text' AND
  -- mandatory partition predicate
  year > 0
GROUP BY geocoded_data["country"]
""")

In [52]:
top_country_pageviews = (
  pageviews_by_country
  .query("pageviews >= 1000")
  .sort_values("pageviews", ascending=False)
  .set_index("country")
)

In [53]:
top_country_pageviews.head(10)

,pageviews
country,
India,3847207
United States,215104
South Africa,211648
Nigeria,61278
Ireland,35472
Tanzania,24366
Pakistan,23804
DR Congo,23506
Uganda,20639


In [31]:
# Adapted from https://github.com/wikimedia/analytics-refinery/blob/master/oozie/unique_devices/per_project_family/monthly/unique_devices_per_project_family_monthly.hql
unique_devices_by_country = hive.run([
"ADD JAR hdfs:///wmf/refinery/current/artifacts/refinery-hive.jar",
"""
CREATE TEMPORARY FUNCTION is_redirect_to_pageview
AS 'org.wikimedia.analytics.refinery.hive.IsRedirectToPageviewUDF'
""",
"""
WITH last_access_dates AS (
  SELECT
    geocoded_data['country'] AS country,
    unix_timestamp(x_analytics_map['WMF-Last-Access-Global'], 'dd-MMM-yyyy') AS last_access_global,
    x_analytics_map['nocookies'] AS nocookies,
    ip,
    user_agent,
    accept_language
  FROM neilpquinn.kaios_wp_webrequest
  WHERE
    x_analytics_map IS NOT NULL AND
    agent_type = 'user' AND
    (
      is_pageview OR
      is_redirect_to_pageview(uri_host, uri_path, uri_query, http_status, content_type, user_agent, x_analytics)
    ) AND
    webrequest_source = 'text' AND
     -- mandatory partition predicate
    year > 0
),
fresh_sessions_aggregated AS (
  SELECT
    country,
    COUNT(1) AS uniques_offset
  FROM (
    SELECT
      HASH(ip, user_agent, accept_language) AS id,
      country,
      SUM(CASE WHEN (nocookies IS NOT NULL) THEN 1 ELSE 0 END)
    FROM
      last_access_dates
    GROUP BY
      hash(ip, user_agent, accept_language),
      country
    -- Only keeping clients having done 1 event without cookies
    HAVING SUM(CASE WHEN (nocookies IS NOT NULL) THEN 1 ELSE 0 END) = 1
  ) fresh_sessions
  GROUP BY country
)
SELECT
  COALESCE(la.country, fresh.country) AS country,
  SUM(
    CASE
      -- last-access-global not set and client accept cookies --> first visit, count
      WHEN (
        la.last_access_global IS NULL AND 
        la.nocookies is NULL
      ) THEN 1
      -- last-access-global set and its date is before month start --> First visit today, count
      WHEN (
        la.last_access_global IS NOT NULL AND
        la.last_access_global < unix_timestamp('2020-07-01', 'yyyy-MM-dd')
      ) THEN 1
      -- Other cases, don't count
      ELSE 0
    END
  ) + COALESCE(fresh.uniques_offset, 0) AS uniques_estimate
FROM last_access_dates AS la
FULL OUTER JOIN fresh_sessions_aggregated AS fresh
ON la.country = fresh.country
GROUP BY
  COALESCE(la.country, fresh.country),
  COALESCE(fresh.uniques_offset, 0)
"""])

In [54]:
unique_devices_by_country = unique_devices_by_country.set_index("country")

In [58]:
top_country_readership = pd.merge(
  top_country_pageviews,
  unique_devices_by_country,
  on="country",
  how="left"
)

In [59]:
top_country_readership.head()

,pageviews,uniques_estimate
country,,
India,3847207,1207720
United States,215104,16452
South Africa,211648,482
Nigeria,61278,2340
Ireland,35472,509


In [61]:
indian_pageviews_by_state = hive.run("""
SELECT
  geocoded_data["subdivision"] AS state,
  COUNT(1) as pageviews
FROM neilpquinn.kaios_wp_webrequest
WHERE
  geocoded_data["country"] = "India" AND
  agent_type = 'user' AND
  is_pageview AND
  webrequest_source = 'text' AND
  -- mandatory partition predicate
  year > 0
GROUP BY geocoded_data["subdivision"]
ORDER BY pageviews DESC
LIMIT 100000
""")

In [62]:
indian_pageviews_by_state.head(10)

,state,pageviews
0,Madhya Pradesh,443518
1,Maharashtra,407684
2,Uttar Pradesh,401183
3,Bihar,366409
4,West Bengal,362807
5,Rajasthan,289302
6,Telangana,238297
7,National Capital Territory of Delhi,216349
8,Tamil Nadu,214613
9,Gujarat,180245


In [64]:
overall_indian_pageviews = hive.run("""
SELECT
  subdivision AS state,
  SUM(view_count) AS total_pageviews
FROM wmf.pageview_hourly
WHERE
  year = 2020 AND month = 7 AND day < 31 AND
  access_method = "mobile web" AND
  country = "India"
GROUP BY subdivision
ORDER BY total_pageviews DESC
LIMIT 100000
""")

In [69]:
indian_state_stats = (
  pd.merge(
    indian_pageviews_by_state.query("pageviews >= 1000"),
    overall_indian_pageviews,
    on="state",
    how="left"
  ).rename({
    "pageviews": "kaios_pageviews",
    "total_pageviews": "pageviews"
  }, axis=1)
  .assign(kaios_view_prop=lambda df: df["kaios_pageviews"] / df["pageviews"])
  .sort_values("state")
  .set_index("state")
  [["kaios_pageviews", "kaios_view_prop"]]
)

In [72]:
indian_state_stats.sort_values("kaios_pageviews", ascending=False).head()

,kaios_pageviews,kaios_view_prop
state,,
Madhya Pradesh,443518,0.017845
Maharashtra,407684,0.004489
Uttar Pradesh,401183,0.010021
Bihar,366409,0.013845
West Bengal,362807,0.008397


# Wikipedia editions

In [74]:
pageviews_per_wp = hive.run("""
SELECT
  language_name as language,
  wiki_lang_part,
  pageviews
FROM (
  SELECT
    normalized_host.project as wiki_lang_part,
    COUNT(1) as pageviews
  FROM neilpquinn.kaios_wp_webrequest
  WHERE
    geocoded_data["country"] = "India" AND
    agent_type = 'user' AND
    is_pageview AND
    webrequest_source = 'text' AND
    -- mandatory partition predicate
    year > 0
  GROUP BY normalized_host.project 
) pv
LEFT JOIN canonical_data.wikis
ON domain_name = CONCAT(wiki_lang_part, ".wikipedia.org")
""")

In [78]:
pageviews_per_wp = (
  pageviews_per_wp
  .set_index("language")
  .sort_values("pageviews", ascending=False)
  [["pageviews"]]
)

In [79]:
pageviews_per_wp.head()

,pageviews
language,
English,2499483
Hindi,812456
Bangla,85020
Tamil,57947
Marathi,56080


# Referrers

In [84]:
hive.run("""
SELECT
  referer_class,
  COUNT(1) as pageviews
FROM neilpquinn.kaios_wp_webrequest
WHERE
  geocoded_data["country"] = "India" AND
  agent_type = 'user' AND
  is_pageview AND
  webrequest_source = 'text' AND
  -- mandatory partition predicate
  year > 0
GROUP BY referer_class
""")

,referer_class,pageviews
0,none,233833
1,external (search engine),2101677
2,external,117645
3,internal,1394052


In [85]:
external_referrers = hive.run("""
SELECT
  PARSE_URL(referer, "HOST") AS referring_domain,
  COUNT(*) as pageviews
FROM neilpquinn.kaios_wp_webrequest
WHERE
  geocoded_data["country"] = "India" AND
  agent_type = 'user' AND
  is_pageview AND
  webrequest_source = 'text' AND
  referer_class LIKE "%external%" AND
  -- mandatory partition predicate
  year > 0
GROUP BY PARSE_URL(referer, "HOST")
HAVING pageviews >= 1000
ORDER BY pageviews DESC
LIMIT 1000000
""")

In [86]:
external_referrers

,referring_domain,pageviews
0,www.google.com,2082422
1,googleweblight.com,109338
2,www.bing.com,6777
3,r.search.yahoo.com,5084
4,www.google.co.in,3449
5,duckduckgo.com,2406


# Top pages

In [89]:
top_pages = hive.run("""
SELECT
  CONCAT(pageview_info["project"], ".org") AS site,
  pageview_info["page_title"] AS page,
  COUNT(*) AS pageviews
FROM neilpquinn.kaios_wp_webrequest
WHERE
  geocoded_data["country"] = "India" AND
  agent_type = 'user' AND
  is_pageview AND
  webrequest_source = 'text' AND
  -- mandatory partition predicate
  year > 0
GROUP BY pageview_info["project"], pageview_info["page_title"]
HAVING pageviews >= 1000
ORDER BY pageviews DESC
LIMIT 1000000
""")

In [90]:
top_pages.head(10)

,site,page,pageviews
0,en.wikipedia.org,Special:Search,182169
1,en.wikipedia.org,Main_Page,100686
2,hi.wikipedia.org,Special:Search,90550
3,en.wikipedia.org,YouTube,53453
4,hi.wikipedia.org,विशेष:खोज,45622
5,en.wikipedia.org,Template:COVID-19_pandemic_data,40666
6,en.wikipedia.org,XXXX,24158
7,hi.wikipedia.org,मुखपृष्ठ,23828
8,hi.wikipedia.org,प्लेयरअननोन्स_बैटलग्राउंड्स,21435
9,en.wikipedia.org,ShareChat,19345


# User agents

In [92]:
user_agents_r = hive.run("""
SELECT
  user_agent,
  COUNT(*) as pageviews
FROM neilpquinn.kaios_wp_webrequest
WHERE
  agent_type = 'user' AND
  is_pageview AND
  webrequest_source = 'text' AND
  -- mandatory partition predicate
  year > 0
GROUP BY user_agent
HAVING pageviews > 1000
ORDER BY pageviews DESC
LIMIT 1000000
""")

In [93]:
user_agents = user_agents_r.copy()

likely_device_names = (
  user_agents
  .replace("\\$", "", regex=True)
  .replace(" LYF_F211S_[\\w-]*", "LYF/F211S", regex=True)
  .replace([
    "Mozilla/5.0\\s*",
    "Mobile;*\\s*",
    "Android;*\\s*",
    "rv:\\s*\\d{1,2}\\.\\d{1};*\\s*",
    "Firefox/\\d{1,2}\\.\\d{1}\\s*",
    "Gecko/\\d{1,2}\\.\\d{1}\\s*",
    "\\(",
    ";*\\s*\\)",
    "(KaiOS|KAIOS)/\\d{1,2}\\.\\d{1}\\.*\\d*\\.*\\d*",
    "/LYF[_-][\\w-]*"
  ], "", regex=True)
  .replace("_", " ", regex=True)
  ["user_agent"]
)

user_agents.insert(1, "likely_device_name", likely_device_names)

user_agents.head()

,user_agent,likely_device_name,pageviews
0,Mozilla/5.0 (Mobile; LYF/F220B/LYF-F220B-003-0...,LYF/F220B,1778594
1,Mozilla/5.0 (Mobile; LYF/F120B/LYF-F120B-001-0...,LYF/F120B,438131
2,Mozilla/5.0 (Mobile; LYF/F90M/LYF-F90M-000-03-...,LYF/F90M,372927
3,Mozilla/5.0 (Mobile; LYF/F271i/LYF_F271i-000-0...,LYF/F271i,263545
4,Mozilla/5.0 (Mobile; LYF/LF-2403N/LYF-LF2403N-...,LYF/LF-2403N,159473
